In [ ]:
import os
import pandas as pd
import requests
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline

Data was downloaded from Kaggle

In [ ]:
os.getcwd()
df = pd.read_csv("data\\train.csv")

Size, data types and summary of the data 

In [ ]:
print(df.info)
df.describe()

The NA's in some columns were replaced based on the data description provided for NA values. Electrical and MasVnrType columns were replaced by the Neighborhood's most common type. Area (in square feet) columns were replaced by the average value in their Neighborhood.

In [ ]:
df.columns[df.isna().any()]
df.loc[df.MasVnrType == None,['MasVnrType', 'MasVnrArea']]
df.loc[df['Electrical'].isna(), :]

In [ ]:
#Lot Frontage
df.loc[df.LotFrontage.isna(),['LotFrontage']]
df['LotFrontage'] = df.groupby(['Neighborhood'])['LotFrontage'].transform(lambda x: x.fillna(x.mean()))
#Mason Veneer Type and Area
df.MasVnrType.unique()
df.loc[df.MasVnrType == 'None',['MasVnrArea','MasVnrType']]
df['MasVnrType'] = df.groupby(['Neighborhood'])['MasVnrType'].transform(lambda x: x.fillna(x.mode().iloc[0]))
df['MasVnrArea'] = df.groupby(['Neighborhood','MasVnrType'])['MasVnrArea'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
#Alley, Fireplace, Pool, Fence
values = {'Alley': 'No Alley', 'FireplaceQu':'No Fireplace','PoolQC':'No Pool','Fence':'No Fence','MiscFeature':'None'}
df.fillna(value = values, inplace = True)
#Basement columns
bsmt_cols = df.columns[df.columns.str.startswith('Bsmt')]
bsmt_obj_cols = bsmt_cols[df[bsmt_cols].dtypes == object]
df[bsmt_obj_cols] = df[bsmt_obj_cols].fillna('No Basement')
#Garage columns
garage_cols = df.columns[df.columns.str.startswith('Garage')]
garage_obj_cols = garage_cols[df[garage_cols].dtypes == object]
df[garage_obj_cols] = df[garage_obj_cols].fillna('No Garage')

In [ ]:
#Electrical
df['Electrical'] = df.groupby(['Neighborhood'])['Electrical'].transform(lambda x: x.fillna(x.mode().iloc[0]))
#GarageYearBuilt
df.GarageYrBlt.fillna(0, inplace = True)
df.columns[df.isna().any()]

Data for US new home mortgage rate was downloaded from the bureau of economic analysis website for years 2000-2010.

In [ ]:
intratedf = pd.read_excel("data\\InterestRates_govinfo.xls", header = None)
intratedf.head(10)

In [ ]:
intratedf1 = intratedf[[0,11]]
intratedf1.columns = ['Year','New_Home_Mortgage_Rate']
intrate = intratedf1.iloc[5:15].reset_index(drop = True)
intrate

In [ ]:
intrate.dtypes
intrate['Year'] = intrate['Year'].astype(float).astype(int)
intrate['New_Home_Mortgage_Rate'] = intrate.New_Home_Mortgage_Rate.astype(float)
intrate

GDP data for Ames, IA was obtained from open data network via API.

In [ ]:
url = "https://odn.data.socrata.com/resource/mkpy-jf3j.json?variable=per_capita_gdp&name=Ames, IA Metro Area"
res = requests.get(url)
json = res.json()
gdpdf = pd.DataFrame(json).sort_values('year').reset_index()
gdp = gdpdf[['year','value']]
gdp.columns = ['Year','Per_Capita_GDP']
#gdp.loc[:,['Year']] = gdp.Year.astype(int)
#gdp.loc[:,['Per_Capita_GDP']] = gdp.Per_Capita_GDP.astype(float)
gdp = gdp.astype(int)
gdp

Population data obtained from US Census Bureau via API for years 2000 - 2010

In [ ]:
url = 'https://api.census.gov/data/2000/pep/int_population?get=GEONAME,POP,DATE_,DATE_DESC&for=place:01855&in=state:19 county:169'
res = requests.get(url)
json = res.json()
json

In [ ]:
popdf1 = pd.DataFrame(json[1:], columns = json[0])
popdf1

In [ ]:
popdf = popdf1[['DATE_DESC','POP']]
popdf

In [ ]:
popdf.loc[:,'Date'] = popdf.DATE_DESC.apply(lambda val: val.split()[0])
Year = pd.to_datetime(popdf.Date).dt.year
Pop = popdf.POP.astype(int)

In [ ]:
pop = pd.concat([Year, Pop], axis = 1)
pop.columns = ['Year', 'Population']
pop

Employment statistics data was downloaded from iowaworkforcedevelopment.org

In [ ]:
empdf = pd.read_csv("data\\Iowa LAUS Data 2020-04_0.csv")
empdf

In [ ]:
empdf.columns
empdf.AREATYNAME.unique()
filterby = (empdf['AREATYNAME'] == 'Cities') & (empdf['AREANAME'] == 'Ames') & (empdf['YEAR'] > 2000) & (empdf['YEAR'] <= 2010)
cols = ['YEAR','MONTH','LABORFORCE','EMP','UNEMP','UNEMPRATE']
empdf1 = empdf.loc[filterby, cols]
empdf1

In [ ]:
empdf1.groupby('YEAR')['MONTH'].nunique()
emp = empdf1.groupby('YEAR').mean().loc[:,['LABORFORCE','UNEMPRATE']].round(1).reset_index()
emp.columns = ['Year','Labor_Force','Unemployment_Rate']
emp.loc[:,['Labor_Force']] = emp.Labor_Force.astype(int)
emp

Median household income data for Story County was downloaded from fred.stlouisfed.org

In [ ]:
incdf = pd.read_csv("data\\Median_Income_crosstab_Full_Data_data.csv")
incdf

In [ ]:
incdf.columns
cols = ['Year', 'Median Household Income']
inc = incdf.loc[:,cols]
inc.columns = ['Year', 'Median_Household_Income']
inc

In [ ]:
dflist = [intrate, gdp, pop, emp, inc]
df_external = reduce(lambda x, y: pd.merge(x, y, on = 'Year'), dflist)
df_external